In [1]:
!pip install xmltodict

In [ ]:
#importing required libraries
import os
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import cv2
import  xml.dom.minidom as minidom
import xmltodict
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision.transforms import transforms
from torchvision import datasets, models, transforms
from glob import glob
from pathlib import Path, PurePath
from PIL import Image
from collections import Counter
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
device

In [ ]:
ANNOTATIONS = "/content/drive/MyDrive/Face Mask Data/annotations"
IMAGES = "/content/drive/MyDrive/Face Mask Data/images"

In [ ]:
#no. of images
images = glob(IMAGES + "/*")
images = sorted(images)
len(images)

In [ ]:
# no of annotations
annotations = glob(f"{ANNOTATIONS}/*")
annotations = sorted(annotations)
len(annotations)

In [ ]:
images[0], annotations[0]

**EDA**

In [ ]:
first_nine_images_shape = []
rows, cols = 3, 3
plt.figure(figsize=(12, 12))

for ind, img in enumerate(images):
    if ind==9:
        break
    im = Image.open(img)
    first_nine_images_shape.append(im.size)
    plt.subplot(rows, cols, ind+1)
    plt.imshow(im)
    im.close()

In [ ]:
first_nine_images_shape

***Resizing images to the same dimensions***

In [ ]:
SIZE = 400

In [ ]:
def visualize_xml(annotations, images):
    for i in range(len(images)):
        img_file = images[i]
        xml_file = annotations[i]
        img = cv2.imread(img_file)
        if img is not None:
            dom = minidom.parse(xml_file)
            root = dom.documentElement
            objects = dom.getElementsByTagName("object")
            print(objects)
            j = 0
            for object in objects:
                bndbox = root.getElementsByTagName('bndbox')[j]
                xmin = bndbox.getElementsByTagName('xmin')[0]
                ymin = bndbox.getElementsByTagName('ymin')[0]
                xmax = bndbox.getElementsByTagName('xmax')[0]
                ymax = bndbox.getElementsByTagName('ymax')[0]

                xmin_data = xmin.childNodes[0].data
                ymin_data = ymin.childNodes[0].data
                xmax_data = xmax.childNodes[0].data
                ymax_data = ymax.childNodes[0].data

                print(object)
                print(xmin_data)
                print(ymin_data)

                j += 1

                cv2.rectangle(img,
                              (int(xmin_data), int(ymin_data)),
                              (int(xmax_data), int(ymax_data)),
                              (55, 255, 155),
                              2
                             )
            cv2.imwrite('/content/drive/MyDrive/Face Mask Data/images/maksssksksss0.png', img)
            break

In [ ]:
for i in range(len(images)):
    img_file = images[i]
    xml_file = annotations[i]
    img = cv2.imread(img_file)
    if img is not None:
        dom = minidom.parse(xml_file)
        root = dom.documentElement
        objects = dom.getElementsByTagName("object")
        j = 0
        for obj in objects:
            bndbox = root.getElementsByTagName('bndbox')[j]
            xmin = bndbox.getElementsByTagName('xmin')[0]
            ymin = bndbox.getElementsByTagName('ymin')[0]
            xmax = bndbox.getElementsByTagName('xmax')[0]
            ymax = bndbox.getElementsByTagName('ymax')[0]

            xmin_data = xmin.childNodes[0].data
            ymin_data = ymin.childNodes[0].data
            xmax_data = xmax.childNodes[0].data
            ymax_data = ymax.childNodes[0].data

            print(obj)
            print(xmin_data)
            print(ymin_data)

            j += 1

            cv2.rectangle(img,
                          (int(xmin_data), int(ymin_data)),
                          (int(xmax_data), int(ymax_data)),
                          (55, 255, 155),
                          2
                         )
        break

In [ ]:
img = Image.open('/content/drive/MyDrive/Face Mask Data/images/maksssksksss0.png')
plt.imshow(img)
img.close()

In [ ]:
with open("/content/drive/MyDrive/Face Mask Data/annotations/maksssksksss0.xml") as fd:
    doc=xmltodict.parse(fd.read())

In [ ]:
doc

In [ ]:
img1 = cv2.imread("/content/drive/MyDrive/Face Mask Data/images/maksssksksss0.png")

In [ ]:
doc['annotation']['object'][0]['bndbox'].values()

In [ ]:
coords = []
labels = []
for i in doc['annotation']['object']:
    bbox = []
    coords.append([int(i['bndbox']['xmin']),
                   int(i['bndbox']['xmax']),
                   int(i['bndbox']['ymin']),
                   int(i['bndbox']['ymax'])])
    labels.append(i['name'])

In [ ]:
coords

In [ ]:
img3 = cv2.rectangle(img1,
              (coords[0][0], coords[0][1]),
              (coords[0][2], coords[0][3]),
              (255, 0, 0),
              2
             )
plt.title("Object Representation")
plt.imshow(img3)

In [ ]:
def plot_coords(coords, color, text, ax):
    x, y, w, h = list(map(int, coords.values()))
    mpatch = mpatches.Rectangle(
        (x ,y),
        w-x, h-y,
        linewidth=1, edgecolor=color, facecolor="none"
    )
    ax.add_patch(mpatch)
    rx, ry = mpatch.get_xy()
    ax.annotate(text, (rx, ry), color=color, weight='bold', fontsize=10, ha='left', va='baseline')

In [ ]:
def annotate_image(img, xml_file):
    with open(xml_file) as f:
        doc = xmltodict.parse(f.read())

    image = plt.imread(img)

    fig, ax = plt.subplots(1)
    ax.axis("off")
    fig.set_size_inches(10, 5)

    temp = doc['annotation']['object']

    if isinstance(temp, list):
        for i in range(len(temp)):
            if temp[i]["name"] == "with_mask":
                plot_coords(temp[i]['bndbox'], 'green', temp[i]["name"],ax)

            if temp[i]['name'] == "mask_weared_incorrect":
                plot_coords(temp[i]['bndbox'], 'yellow', temp[i]["name"], ax)

            if temp[i]['name'] == "without_mask":
                plot_coords(temp[i]['bndbox'], 'red', temp[i]["name"], ax)
    else:
        x, y, w, h = list(map(int, temp["bndbox"].values()))
        edgecolor = {"with_mask": "g", "without_mask": "r", "mask_weared_incorrect": "y"}
        mpatch = mpatches.Rectangle(
            (x, y),
            w-x, h-y,
            linewidth=1,
            edgecolor=edgecolor[temp["name"]],
            facecolor="none")
        ax.add_patch(mpatch)
        rx, ry = mpatch.get_xy()
        ax.annotate(temp["name"], (rx, ry),
                    color=edgecolor[temp["name"]], weight='bold',
                    fontsize=10, ha='left', va='baseline')
    ax.imshow(image)

In [ ]:
for i in range(1, 8):
    annotate_image(images[i], annotations[i])

In [ ]:
immg = annotate_image("/content/drive/MyDrive/Face Mask Data/images/maksssksksss10.png",
              "//content/drive/MyDrive/Face Mask Data/annotations/maksssksksss10.xml")

In [ ]:
annotated_dataset_folder = "/content/drive/MyDrive/Face Mask Data/annotated_dataset"
os.mkdir(annotated_dataset_folder)

In [ ]:
{'annotation': {'folder': 'images',
  'filename': 'maksssksksss0.png',
  'size': {'width': '512', 'height': '366', 'depth': '3'},
  'segmented': '0',
  'object': [{'name': 'without_mask',
    'pose': 'Unspecified',
    'truncated': '0',
    'occluded': '0',
    'difficult': '0',
    'bndbox': {'xmin': '79', 'ymin': '105', 'xmax': '109', 'ymax': '142'}},
   {'name': 'with_mask',
    'pose': 'Unspecified',
    'truncated': '0',
    'occluded': '0',
    'difficult': '0',
    'bndbox': {'xmin': '185', 'ymin': '100', 'xmax': '226', 'ymax': '144'}},
   {'name': 'without_mask',
    'pose': 'Unspecified',
    'truncated': '0',
    'occluded': '0',
    'difficult': '0',
    'bndbox': {'xmin': '325', 'ymin': '90', 'xmax': '360', 'ymax': '141'}}]}}

In [ ]:
classes = ['with_mask', 'without_mask', 'mask_weared_incorrect']

In [ ]:
for i in classes:
    os.mkdir(os.path.join(annotated_dataset_folder, i))

In [ ]:
def plot_images(img, vertices, image_path, width, height):
    imgg = cv2.imread(img)
    imgg = cv2.resize(imgg, (width, height), interpolation=cv2.INTER_AREA)
    for i in range(len(vertices)):
        imgg = cv2.rectangle(imgg,
                             (int(vertices[0]), int(vertices[1])),
                             (int(vertices[2]), int(vertices[3])),
                             (255, 0, 0),
                             2)
    cv2.imwrite(image_path, imgg)

In [ ]:
class_dict = {"with_mask": 0, "without_mask": 1, "mask_weared_incorrect": 2}

In [ ]:
def create_df_uniclass(xmls, new_height, new_width, dataset_folderannotated_dataset_folder):

    df = {
#         "image_array": [],
        "image_path": [],
        "new_image_path": [],
        "og_width": [],
        "og_height": [],
        "new_width": [],
        "new_height": [],
        "xmin": [], # [ [ [x1,y1,x2,y2], [x1,x2,y1,y2] ], [ [], [] ], [ [] ] ]
        "xmax": [],
        "ymin": [],
        "ymax": [],
        "classes": [],
        "vertices": [],
        "labels": [] # [ [1, 2, 3], [] ]
    }
    for xml_file in xmls:
        with open(xml_file) as f:
            doc = xmltodict.parse(f.read())

        img_name = "/content/drive/MyDrive/Face Mask Data/images/" + doc['annotation']['filename']
        width = int(doc['annotation']['size']['width'])
        height = int(doc['annotation']['size']['height'])
        temp = doc['annotation']['object']


        if isinstance(temp, list):

            for i in range(len(temp)):
                df['image_path'].append(img_name)
                df['og_width'].append(width)
                df['og_height'].append(height)
                df['new_width'].append(new_width)
                df['new_height'].append(new_height)

                x1, y1, x2, y2 = list(map(int, temp[i]['bndbox'].values()))
                nv_x1, nv_y1, nv_x2, nv_y2 = x1 / width, y1 / height, x2 / width, y2 / height
                n_x1, n_y1, n_x2, n_y2 = int(nv_x1 * new_width), int(nv_y1 * new_height), int(nv_x2 * new_width), int(nv_y2 * new_height)
                fn_x1, fn_y1, fn_x2, fn_y2 = nv_x1 * new_width, nv_y1 * new_height, nv_x2 * new_width, nv_y2 * new_height

                dataset_folder = os.path.join(annotated_dataset_folder, temp[i]["name"])
                new_image_path = os.path.join(dataset_folder, Path(img_name).name)
                df['new_image_path'].append(new_image_path)


                df['labels'].append(temp[i]["name"])
                df['xmax'].append(n_x2)
                df['xmin'].append(n_x1)
                df['ymax'].append(n_y2)
                df['ymin'].append(n_y1)
                df['classes'].append(class_dict[temp[i]["name"]])
                df['vertices'].append([fn_x1, fn_y1, fn_x2, fn_y2])
                plot_images(img_name, [n_x1, n_y1, n_x2, n_y2], new_image_path, new_width, new_height)

        else:
            df['image_path'].append(img_name)
            df['og_width'].append(width)
            df['og_height'].append(height)
            df['new_width'].append(new_width)
            df['new_height'].append(new_height)
            x1, y1, x2, y2 = list(map(int, temp['bndbox'].values()))
            nv_x1, nv_y1, nv_x2, nv_y2 = x1 / width, y1 / height, x2 / width, y2 / height
            n_x1, n_y1, n_x2, n_y2 = int(nv_x1 * new_width), int(nv_y1 * new_height), int(nv_x2 * new_width), int(nv_y2 * new_height)
            fn_x1, fn_y1, fn_x2, fn_y2 = nv_x1 * new_width, nv_y1 * new_height, nv_x2 * new_width, nv_y2 * new_height
            dataset_folder = os.path.join(annotated_dataset_folder, temp["name"])
            new_image_path = os.path.join(dataset_folder, Path(img_name).name)
            df['new_image_path'].append(new_image_path)


            df['labels'].append(temp["name"])
            df['xmax'].append(n_x2)
            df['xmin'].append(n_x1)
            df['ymax'].append(n_y2)
            df['ymin'].append(n_y1)
            df['classes'].append(class_dict[temp["name"]])
            df['vertices'].append([fn_x1, fn_y1, fn_x2, fn_y2])
            plot_images(img_name, [n_x1, n_y1, n_x2, n_y2], new_image_path, new_width, new_height)


    return df

In [ ]:
IMG_WIDTH, IMG_HEIGHT = 224, 224
df1 = create_df_uniclass(annotations, IMG_HEIGHT, IMG_WIDTH, annotated_dataset_folder)

In [ ]:
df1 = pd.DataFrame(df1)

In [ ]:
df1.head()

In [ ]:
classes = list((df1['labels'].value_counts()).index)
classes

In [ ]:
train_df, test_df = train_test_split(df1, stratify=df1['classes'], test_size=0.1)

In [ ]:
train_images = train_df['new_image_path'].values
test_images = test_df['new_image_path'].values

train_labels = torch.from_numpy(train_df['classes'].values)
test_labels = torch.from_numpy(test_df['classes'].values)

train_bbox = torch.from_numpy(train_df[['xmin', 'xmax', 'ymin', 'ymax']].values)
test_bbox = torch.from_numpy(test_df[['xmin', 'xmax', 'ymin', 'ymax']].values)

Data Loader

In [ ]:
annotated_dataset_folder

In [ ]:
BATCH_SIZE = 32

In [ ]:
train = datasets.ImageFolder(annotated_dataset_folder,
                   transform = transforms.Compose([
                       transforms.Resize(IMG_WIDTH),
                       transforms.RandomCrop(IMG_WIDTH),
                       transforms.ToTensor()
                   ]))
train_data_loader = torch.utils.data.DataLoader(train, 32, shuffle=True, num_workers=3, pin_memory=True)

In [ ]:
def get_mean_std(data_loader):
    sum_, squared_sum, batches = 0, 0, 0
    for data, _ in data_loader:
        sum_ += torch.mean(data, dim=([0, 2, 3]))
        squared_sum += torch.mean(data ** 2, dim=([0, 2, 3]))
        batches += 1

    mean = sum_ / batches
    std = (squared_sum / batches - mean ** 2) ** 0.5
    return mean, std

In [ ]:
mean, std = get_mean_std(train_data_loader)
mean, std

In [ ]:
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(IMG_HEIGHT),
    transforms.RandomCrop(IMG_WIDTH),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

In [ ]:
test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(IMG_HEIGHT),
    transforms.CenterCrop(IMG_HEIGHT),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)])

In [ ]:
class CustomTensorDataset(Dataset):

    def __init__(self, tensors, transforms=None):
        self.tensors = tensors
        self.transforms = transforms

    def __getitem__(self, index):
#         image = self.tensors[0][index]
        label = self.tensors[1][index]
        bbox = self.tensors[2][index]

        image = cv2.imread(self.tensors[0][index])
        image = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1)
#         image = image.permute(2, 0, 1)

        if self.transforms:
            image = self.transforms(image)

        return (image, label, bbox)

    def __len__(self):
        return self.tensors[0].shape[0]

In [ ]:
trainset = CustomTensorDataset((train_images, train_labels, train_bbox),transforms=train_transform)
testset = CustomTensorDataset((test_images, test_labels, test_bbox),transforms=test_transform)

In [ ]:
train_data_loader = torch.utils.data.DataLoader(trainset, BATCH_SIZE, shuffle=True)
test_data_loader = torch.utils.data.DataLoader(testset, BATCH_SIZE, shuffle=True)

CNN

In [ ]:
class ObjectDetection(nn.Module):

    def __init__(self, in_channels=3, out_channels_cnn=3, bboxes=4):
        super().__init__()
        hidden_channel1 = 32
        hidden_channel2 = 64
        hidden_channel3 = 128
        kernel_size = 3
        stride = 1
        padding = 1
        self.conv1 = nn.Conv2d(in_channels=in_channels,
                               out_channels=hidden_channel1,
                               kernel_size=kernel_size,
                               stride=stride,
                               padding=padding)
        self.conv2 = nn.Conv2d(in_channels=hidden_channel1,
                               out_channels=hidden_channel2,
                               kernel_size=kernel_size,
                               stride=stride,
                               padding=padding)
        self.conv3 = nn.Conv2d(in_channels=hidden_channel2,
                               out_channels=hidden_channel2,
                               kernel_size=kernel_size,
                               stride=stride,
                               padding=padding)
        self.conv4 = nn.Conv2d(in_channels=hidden_channel2,
                               out_channels=hidden_channel3,
                               kernel_size=kernel_size,
                               stride=stride,
                               padding=padding)
        self.conv5 = nn.Conv2d(in_channels=hidden_channel3,
                               out_channels=hidden_channel3,
                               kernel_size=kernel_size,
                               stride=stride,
                               padding=padding)

        self.batchnorm1 = nn.BatchNorm2d(hidden_channel1)
        self.batchnorm2 = nn.BatchNorm2d(hidden_channel2)
        self.batchnorm3 = nn.BatchNorm2d(hidden_channel3)
        self.maxpool = nn.MaxPool2d(kernel_size=2)
        self.relu = nn.ReLU()
        self.fc = nn.Flatten()
        self.cnn_layer = nn.Linear(7*7*128, out_channels_cnn)
        self.regressor = nn.Linear(7*7*128, bboxes)

    def cnn_layers(self, x):
        x = self.relu(x)
        x = self.maxpool(x)
        return x

    def feature_extractor(self, x):
        x = self.conv1(x)
        x = self.batchnorm1(x)
        x = self.cnn_layers(x)

        x = self.conv2(x)
        x = self.batchnorm2(x)
        x = self.cnn_layers(x)

        x = self.conv3(x)
        x = self.batchnorm2(x)
        x = self.cnn_layers(x)

        x = self.conv4(x)
        x = self.batchnorm3(x)
        x = self.cnn_layers(x)

        x = self.conv5(x)
        x = self.batchnorm3(x)
        x = self.cnn_layers(x)

        x = self.fc(x)
        return x

    def forward(self, x):
        x = self.feature_extractor(x)
        classifier_op = self.cnn_layer(x)
        regressor_op = self.regressor(x)
        return (regressor_op, classifier_op)

In [ ]:
model = ObjectDetection(in_channels=3, out_channels_cnn=3, bboxes=4)
model = model.to(device)

In [ ]:
model

In [ ]:
classification_loss_fn = nn.CrossEntropyLoss()
bbox_loss_fn = nn.MSELoss()

In [ ]:
LR = 0.001
opt = torch.optim.Adam(model.parameters(), lr=LR)

In [ ]:
EPOCHS = 10
train_loss = []
train_accuracy = []
test_loss = []
test_accuracy = []

In [ ]:
for epoch in tqdm(range(EPOCHS)):
    correct = 0
    iterations = 0
    iter_loss = 0
    model.train()
    for i, (images, labels, bbox) in enumerate(train_data_loader):
        images = images.to(device)
        labels = labels.to(device)
        bbox = bbox.to(device)
        regressor, classifier = model(images)

        _, predicted = torch.max(classifier, 1)
        predicted_bbox = bbox + regressor

        clf_loss = classification_loss_fn(classifier, labels)
        reg_loss = bbox_loss_fn(predicted_bbox, bbox)

        total_loss = (clf_loss + reg_loss).clone().detach().requires_grad_(True)
        opt.zero_grad()
        total_loss.backward()
        opt.step()

        iter_loss += total_loss.item()
        correct += (predicted == labels).sum().item()
        iterations += 1

    train_loss.append(iter_loss / iterations)
    train_accuracy.append((100 * correct / len(trainset)))
    print(f"Epoch [{epoch + 1} / {EPOCHS}], Training Loss: {train_loss[-1]:.3f}, Training Accuracy: {train_accuracy[-1]:.3f}")

In [ ]:
train_loss